In [1]:
import os
import pandas as pd
#from Bio import SeqIO
#from Bio import Seq

os.getcwd()
os.chdir('/Volumes/AHN/captive_ape_microbiome')
from ete3 import Tree


tree_file = "results/gyrb_all/Bt/phylogeny/ASVs_filtered_ref.fasta.aln.tree"
tree = Tree(tree_file)

Process fna supplement from moeller et al. 2016 to create phylogeny in order to pick out the subtrees that correspond to co-diversifying clades. Transalign generates error from some seqs because the translated seqs have stop codons.  

In [3]:
%%bash
cd /Volumes/AHN/captive_ape_microbiome
mkdir -pv results/moeller_sup
cutadapt  -g CGGAGGTAARTTCGAYAAAGG  --overlap 21 -e .15 --discard-untrimmed  -o  results/moeller_sup/Bacteroidaceae.fna  data/moeller_sup/Bacteroidaceae.fna

transeq -frame 2 -sequence results/moeller_sup/Bacteroidaceae.fna  -outseq results/moeller_sup/Bacteroidaceae.faa

cp results/gyrb_bt_gtdbtk_ref/alignment/gtdbtk_gyrb_Bt.faa results/moeller_sup/gtdbtk_gyrb_Bt.faa
cp results/gyrb_bt_gtdbtk_ref/alignment/gtdbtk_gyrb_Bt.fasta results/moeller_sup/gtdbtk_gyrb_Bt.fasta

cat results/moeller_sup/gtdbtk_gyrb_Bt.faa results/moeller_sup/Bacteroidaceae.faa > results/moeller_sup/Bacteroidaceae_ref.faa
cat results/moeller_sup/gtdbtk_gyrb_Bt.fasta results/moeller_sup/Bacteroidaceae.fna > results/moeller_sup/Bacteroidaceae_ref.fna

mafft --auto --quiet results/moeller_sup/Bacteroidaceae_ref.faa > results/moeller_sup/Bacteroidaceae_ref.faa.aln
tranalign -asequence  results/moeller_sup/Bacteroidaceae_ref.fna -bsequence results/moeller_sup/Bacteroidaceae_ref.faa.aln -outseq results/moeller_sup/Bacteroidaceae_ref.fna.aln


This is cutadapt 2.5 with Python 3.7.7
Command line parameters: -g CGGAGGTAARTTCGAYAAAGG --overlap 21 -e .15 --discard-untrimmed -o results/moeller_sup/Bacteroidaceae.fna data/moeller_sup/Bacteroidaceae.fna
Processing reads on 1 core in single-end mode ...
Finished in 0.69 s (954 us/read; 0.06 M reads/minute).

=== Summary ===

Total reads processed:                     724
Reads with adapters:                       724 (100.0%)
Reads written (passing filters):           724 (100.0%)

Total basepairs processed:       217,200 bp
Total written (filtered):        200,991 bp (92.5%)

=== Adapter 1 ===

Sequence: CGGAGGTAARTTCGAYAAAGG; Type: regular 5'; Length: 21; Trimmed: 724 times.

No. of allowed errors:
0-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20-21 bp: 3

Overview of removed sequences
length	count	expect	max.err	error counts
21	194	0.0	3	194
22	192	0.0	3	192
23	201	0.0	3	201
24	137	0.0	3	137


Translate nucleic acid sequences
Generate an alignment of nucleic coding regions from aligned proteins
Error: Guide protein sequence Gorilla3374716_2 not found in nucleic sequence Gorilla3374716
Error: Guide protein sequence Human3058374_2 not found in nucleic sequence Human3058374
Error: Guide protein sequence Human9403092_2 not found in nucleic sequence Human9403092
Error: Guide protein sequence Chimp10413648_2 not found in nucleic sequence Chimp10413648
Error: Guide protein sequence Chimp10757854_2 not found in nucleic sequence Chimp10757854
Error: Guide protein sequence Chimp10796222_2 not found in nucleic sequence Chimp10796222
Error: Guide protein sequence Chimp11562025_2 not found in nucleic sequence Chimp11562025
Error: Guide protein sequence Chimp3146215_2 not found in nucleic sequence Chimp3146215
Error: Guide protein sequence Chimp8175258_2 not found in nucleic sequence Chimp8175258
Error: Guide protein sequence Bonobo2919026_2 not found in nucleic sequence Bonobo2919026
Err

In [ ]:

cp results/moeller_sup/Bacteroidaceae_codiv.fna results/gyrb_all/Bt/moeller_sup/
cd results/gyrb_all/Bt

#make blastdb
makeblastdb -in moeller_sup/Bacteroidaceae_codiv.fna -dbtype nucl

#blast moeller co-div seqs
blastn -query ASVs_filtered.fasta -db moeller_sup/Bacteroidaceae_codiv.fna \
-outfmt "7 qseqid salltitles sseqid pident length qlen evalue" \
-out moeller_sup/ASVs_filtered_blastout.txt \
-max_target_seqs 5